In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from numpy import sqrt
import pandas as pd
import scipy.constants as const
from scipy.optimize import curve_fit                        # Funktionsfit:     popt, pcov = curve_fit(func, xdata, ydata) 
from uncertainties import ufloat                            # Fehler:           fehlerwert =  ulfaot(x, err)
from uncertainties import unumpy as unp 
from uncertainties.unumpy import uarray                     # Array von Fehler: fehlerarray =  uarray(array, errarray)
from uncertainties.unumpy import (nominal_values as noms,   # Wert:             noms(fehlerwert) = x
                                  std_devs as stds)         # Abweichung:       stds(fehlerarray) = errarray

Tabelle erstellen:

In [12]:
md = pd.read_csv('tables/md.csv')
md85 = md.iloc[:, [0,1,2]]
md87 = md.iloc[:, [0,3,4]]
print(md85.to_latex(index = False, column_format= "c c c", decimal=',')) 
print(md87.to_latex(index = False, column_format= "c c c", decimal=',')) 

\begin{tabular}{c c c}
\toprule
 f/kHz &  I\_1/mA &  I\_1H/mA \\
\midrule
   100 &     660 &        0 \\
   200 &     652 &       16 \\
   300 &     486 &       44 \\
   400 &     409 &       66 \\
   500 &     375 &       84 \\
   600 &     224 &      112 \\
   700 &     164 &      132 \\
   800 &     295 &      140 \\
   900 &     269 &      158 \\
  1000 &     590 &      158 \\
\bottomrule
\end{tabular}

\begin{tabular}{c c c}
\toprule
 f/kHz &  I\_2/mA &  I\_2H/mA \\
\midrule
   100 &     764 &        0 \\
   200 &     888 &       16 \\
   300 &     842 &       44 \\
   400 &     883 &       66 \\
   500 &     967 &       84 \\
   600 &     938 &      112 \\
   700 &     993 &      132 \\
   800 &     629 &      184 \\
   900 &     357 &      226 \\
  1000 &     586 &      236 \\
\bottomrule
\end{tabular}



Daten auswerten:

In [ ]:
np.savetxt('tables/a.txt', md.values, header='f/Hz U/V', fmt='%.3f')
f, U = np.genfromtxt('tables/a.txt', unpack=True, skip_header=1)

# Ausgleichsfunktion
def g(f, T):
    w = 2 * np.pi * f
    return 1 / (np.sqrt(1 + w**2 * T**2))

para, pcov = curve_fit(g, f, U)
pcov = np.sqrt(np.diag(pcov))
a = para
fa = pcov 

Fehler berechnen:

In [ ]:
ua = ufloat(a, fa) 
print('T = (%.3f ± %.3f)' % (noms(ua), stds(ua)))

Plot erstellen:

In [ ]:

plt.plot(f, Uc, 'xr', markersize=6 , label = 'Messdaten', alpha=0.5)
xx = np.linspace(100, 150000, 10**4)
plt.plot(xx, g(xx, a), '-b', linewidth = 1, label = 'Ausgleichsfunktion')

plt.xlabel(r'$f \, / \, \mathrm{Hz}$')
plt.ylabel(r'$A(\omega) \, / \, U_0$')
plt.xscale('log')                       # axis scale 
plt.legend(loc="best")                  # legend position
plt.grid(True)                          # grid style
plt.xlim(200, 150000)                   # limitation of visible scale in plot